In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import os, os.path
import sys
sys.path.insert(0,'../processing/')
sys.path.insert(0,'../')
from datasets import sythtextprovider
import load_batch
from nets import txtbox_300, textbox_common, np_methods
#from processing import image_processing
from image_processing2 import *
from processing import txt_preprocessing
import tf_utils
import time
slim = tf.contrib.slim
import load_batch
import numpy as np
import skimage.io as skio
from processing import tf_image
import cv2

In [2]:
isess = tf.InteractiveSession()

In [ ]:
with tf.Graph().as_default(): 
    text_net = txtbox_300.TextboxNet()
    text_shape = text_net.params.img_shape
    print 'text_shape '+  str(text_shape)
    text_anchors = text_net.anchors(text_shape)

    b_image, b_glocalisations, b_gscores = \
        load_batch.get_batch('../data/sythtext/',
                             3,
                             1,
                             text_shape,
                             text_net,
                             text_anchors,
                             2,
                             file_pattern = '*.tfrecord',
                             is_training = True,
                             shuffe = False)
    localisations, logits, end_points = \
                        text_net.net(b_image, is_training=True, use_batch=False)
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        with slim.queues.QueueRunners(sess):
            for i in xrange(1):
                

In [3]:
def pad2d(inputs,
          pad=(0, 0),
          mode='CONSTANT',
          data_format='NHWC',
          trainable=True,
          scope=None):
    """2D Padding layer, adding a symmetric padding to H and W dimensions.

    Aims to mimic padding in Caffe and MXNet, helping the port of models to
    TensorFlow. Tries to follow the naming convention of `tf.contrib.layers`.

    Args:
      inputs: 4D input Tensor;
      pad: 2-Tuple with padding values for H and W dimensions;
      mode: Padding mode. C.f. `tf.pad`
      data_format:  NHWC or NCHW data format.
    """
    with tf.name_scope(scope, 'pad2d', [inputs]):
        # Padding shape.
        if data_format == 'NHWC':
            paddings = [[0, 0], [pad[0], pad[0]], [pad[1], pad[1]], [0, 0]]
        elif data_format == 'NCHW':
            paddings = [[0, 0], [0, 0], [pad[0], pad[0]], [pad[1], pad[1]]]
        net = tf.pad(inputs, paddings, mode=mode)
        return net
    
def conv2d(inputs, out, kernel_size, scope,stride=1,activation_fn=tf.nn.relu, 
			padding = 'SAME', use_batch=False, batch_norm_params={}, rate = 1):
	if use_batch:
		net = slim.conv2d(inputs, out, kernel_size, stride=stride ,scope=scope, normalizer_fn=slim.batch_norm, 
			  normalizer_params=batch_norm_params, activation_fn=activation_fn ,padding = padding, rate = rate)
	else:
		net = slim.conv2d(inputs, out, kernel_size, stride=stride, scope=scope, activation_fn=activation_fn,padding = padding, rate = rate)
	return net

In [16]:
scope='text_box_300'
inputs = tf.placeholder(shape=(None,512,512,3),dtype=tf.float32)
use_batch = True
is_training= True
batch_norm_params = {
  # Decay for the moving averages.
  'decay': 0.9997,
  # epsilon to prevent 0s in variance.
  'epsilon': 0.001,
  'is_training': is_training,
}
end_points = {}
with tf.variable_scope(scope, 'text_box_300', [inputs], reuse=True):
    # Original VGG-16 blocks.
    net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
    end_points['conv1'] = net
    net = slim.max_pool2d(net, [2, 2], scope='pool1')
    # Block 2.
    net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
    end_points['conv2'] = net # 150,150 128
    net = slim.max_pool2d(net, [2, 2], scope='pool2')
    # Block 3. # 75 75 256
    net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
    end_points['conv3'] = net
    net = slim.max_pool2d(net, [2, 2], scope='pool3',padding='SAME')
    # Block 4. # 38 38 512
    net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
    end_points['conv4'] = net
    #net = slim.max_pool2d(net, [2, 2],scope='pool4')
    net = slim.max_pool2d(net, [2, 2], stride=2, scope='pool4')
    # Block 5. # 19 19 512
    #net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
    net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
    end_points['conv5'] = net
    net = slim.max_pool2d(net, [3, 3], stride=1, scope='pool5',padding='SAME')

    # Additional SSD blocks.
    # Block 6: let's dilate the hell out of it!
    #net = slim.conv2d(net, 1024, [3, 3], scope='conv6')
    #net = conv2d(net, 1024, [3,3], scope='conv6',rate=1, use_batch=use_batch, batch_norm_params= batch_norm_params)
    net = conv2d(net, 1024, [3,3], scope='conv6',rate=6, use_batch=use_batch, batch_norm_params= batch_norm_params)
    end_points['conv6'] = net
    # Block 7: 1x1 conv. Because the fuck.
    #net = slim.conv2d(net, 1024, [1, 1], scope='conv7')
    net = conv2d(net, 1024, [1, 1], scope='conv7',use_batch=use_batch, batch_norm_params= batch_norm_params)
    end_points['conv7'] = net
    # Block 8/9/10/11: 1x1 and 3x3 convolutions stride 2 (except lasts).
    end_point = 'conv8'
    with tf.variable_scope(end_point):
        #net = slim.conv2d(net, 256, [1, 1], scope='conv1x1')
        #net = slim.conv2d(net, 512, [3, 3], stride=2, scope='conv3x3')
        net = conv2d(net, 256, [1, 1], scope='conv1x1',use_batch=use_batch, batch_norm_params=batch_norm_params)
        net = conv2d(net, 512, [3, 3], stride=2, scope='conv3x3',use_batch=use_batch, batch_norm_params=batch_norm_params)	
    end_points[end_point] = net
    end_point = 'conv9'
    with tf.variable_scope(end_point):
        net = conv2d(net, 128, [1, 1], scope='conv1x1', use_batch=use_batch, batch_norm_params=batch_norm_params)
        net = conv2d(net, 256, [3, 3], stride=2, scope='conv3x3',use_batch=use_batch, batch_norm_params=batch_norm_params)
    end_points[end_point] = net
    end_point = 'conv10'
    with tf.variable_scope(end_point):
        net = conv2d(net, 128, [1, 1], scope='conv1x1',use_batch=use_batch, batch_norm_params=batch_norm_params)
        net = conv2d(net, 256, [3, 3], stride=2, scope='conv3x3',use_batch=use_batch, batch_norm_params=batch_norm_params)
    end_points[end_point] = net
    end_point = 'global'
    with tf.variable_scope(end_point):
        net = conv2d(net, 128, [1, 1], scope='conv1x1',use_batch=use_batch, batch_norm_params=batch_norm_params)
        net = conv2d(net, 256, [3, 3], scope='conv3x3', padding='VALID',use_batch=use_batch, batch_norm_params=batch_norm_params)
    end_points[end_point] = net

In [17]:
end_points


{'conv1': <tf.Tensor 'text_box_300_7/conv1/conv1_2/Relu:0' shape=(?, 512, 512, 64) dtype=float32>,
 'conv10': <tf.Tensor 'text_box_300_7/conv10/conv3x3/Relu:0' shape=(?, 4, 4, 256) dtype=float32>,
 'conv2': <tf.Tensor 'text_box_300_7/conv2/conv2_2/Relu:0' shape=(?, 256, 256, 128) dtype=float32>,
 'conv3': <tf.Tensor 'text_box_300_7/conv3/conv3_3/Relu:0' shape=(?, 128, 128, 256) dtype=float32>,
 'conv4': <tf.Tensor 'text_box_300_7/conv4/conv4_3/Relu:0' shape=(?, 64, 64, 512) dtype=float32>,
 'conv5': <tf.Tensor 'text_box_300_7/conv5/conv5_3/Relu:0' shape=(?, 32, 32, 512) dtype=float32>,
 'conv6': <tf.Tensor 'text_box_300_7/conv6/Relu:0' shape=(?, 32, 32, 1024) dtype=float32>,
 'conv7': <tf.Tensor 'text_box_300_7/conv7/Relu:0' shape=(?, 32, 32, 1024) dtype=float32>,
 'conv8': <tf.Tensor 'text_box_300_7/conv8/conv3x3/Relu:0' shape=(?, 16, 16, 512) dtype=float32>,
 'conv9': <tf.Tensor 'text_box_300_7/conv9/conv3x3/Relu:0' shape=(?, 8, 8, 256) dtype=float32>,
 'global': <tf.Tensor 'text_box